In [21]:
import os
from typing import Dict, Any, Optional

from dotenv import load_dotenv
from neo4j import GraphDatabase

# LangChain imports
from langchain_neo4j import Neo4jVector, Neo4jGraph
from langchain_huggingface import HuggingFaceEmbeddings

In [22]:
# Neo4j Configuration
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7689")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "password123")

# Configuration de l'embedding (doit correspondre à celle utilisée dans 003_index_content.py)
EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_MODEL_NAME", "Qwen/Qwen3-Embedding-0.6B")
EMBEDDING_MODEL_KWARGS = {"device": "cpu"}  # Utiliser "cuda" si GPU disponible
EMBED_ENCODE_KWARGS = {"normalize_embeddings": True}  # Normaliser pour la similarité cosinus

# Configuration de l'index vectoriel
VECTOR_INDEX_NAME = os.getenv("VECTOR_INDEX_NAME", "content_vector_qwen_index")
EMBEDDING_NODE_PROPERTY = "embedding_qwen"  # Nom de la propriété où l'embedding est stocké

# Nombre de résultats par défaut
DEFAULT_K = 20


In [23]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USER, password=NEO4J_PASSWORD, enhanced_schema=True)

In [25]:
graph.query("SHOW INDEXES"), len(graph.query("SHOW INDEXES"))

([{'id': 16,
   'name': 'content_chunk_order',
   'state': 'ONLINE',
   'populationPercent': 100.0,
   'type': 'RANGE',
   'entityType': 'NODE',
   'labelsOrTypes': ['Content'],
   'properties': ['chunk_order'],
   'indexProvider': 'range-1.0',
   'owningConstraint': None,
   'lastRead': None,
   'readCount': 0},
  {'id': 15,
   'name': 'content_source_url',
   'state': 'ONLINE',
   'populationPercent': 100.0,
   'type': 'RANGE',
   'entityType': 'NODE',
   'labelsOrTypes': ['Content'],
   'properties': ['source_url'],
   'indexProvider': 'range-1.0',
   'owningConstraint': None,
   'lastRead': neo4j.time.DateTime(2025, 12, 20, 2, 58, 12, 825000000, tzinfo=<UTC>),
   'readCount': 252},
  {'id': 14,
   'name': 'content_type',
   'state': 'ONLINE',
   'populationPercent': 100.0,
   'type': 'RANGE',
   'entityType': 'NODE',
   'labelsOrTypes': ['Content'],
   'properties': ['content_type'],
   'indexProvider': 'range-1.0',
   'owningConstraint': None,
   'lastRead': None,
   'readCount': 

In [26]:
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs=EMBEDDING_MODEL_KWARGS,
    encode_kwargs=EMBED_ENCODE_KWARGS
)

In [27]:
keyword_index_name = "keyword"

In [28]:
vector_store = Neo4jVector.from_existing_index(
    embedding=embeddings,
    url=NEO4J_URI,
    username=NEO4J_USER,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label="Content",
    text_node_property="chunk_content",  # Utiliser chunk_content au lieu de text
    embedding_node_property=EMBEDDING_NODE_PROPERTY,
    #keyword_index_name=keyword_index_name,
    #search_type="hybrid",
    #retrieval_query=retrieval_query,  # Requête personnalisée pour mapper chunk_content
)

In [29]:
query = "Prof. Dipl. Ing. Gerald Lange"
k=20
results_with_scores = vector_store.similarity_search_with_score(query, k=k)

In [38]:
results_with_scores

[(Document(metadata={'source_url': 'https://www.fh-swf.de/de/ueber_uns/standorte_4/hagen_4/fb_tbw_1/dozentinnen_6/wiss__mitarbeiter/goedde/index.php', 'chunk_order': 1, 'total_chunks': 1, 'source_title': 'Dipl. Wirt-Ing (FH) Elmar Gödde', 'chunk_size': 631, 'chunk_metadata': '{}', 'source_meta': 'Dipl. Wirt-Ing (FH) Elmar Gödde', 'embedding': [-0.09752225130796432, 0.09845636785030365, 0.03401922062039375, -0.030558744445443153, 0.013246753253042698, -0.006533298175781965, 0.01247753482311964, 0.10406934469938278, 0.07162763923406601, -0.020716547966003418, 0.05001745745539665, -0.021228773519396782, 0.03464558348059654, -0.04447374492883682, -0.034983858466148376, -0.036045853048563004, -0.11395073682069778, 0.00140504096634686, -0.07911323010921478, 0.06107821315526962, 0.009261520579457283, 0.0056709847413003445, -0.049174997955560684, 0.029708947986364365, 0.030586527660489082, 0.013843141496181488, -0.03295089676976204, -0.03472418710589409, 0.037828754633665085, -0.03189148753881